In [1]:
import torch
import os
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split, Dataset
from PIL import Image

In [ ]:
class CatDogDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.file_names = [f for f in os.listdir(data_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        label = 1 if 'dog' in file_name else 0  
        img_path = os.path.join(self.data_dir, file_name)
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose([transforms.Resize((128, 128)), transforms.ToTensor(), transforms.Normalize(mean=[0.5], std=[0.5])])
data_dir = r'D:\AI\study1\data\train1\train'
dataset = CatDogDataset(data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
import torch.nn as nn
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64,2)
        )
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [4]:
import torch.optim as optim
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN_model().to(device)
criterion = nn.CrossEntropyLoss()
optimzer = optim.Adam(model.parameters(), lr=0.001)

def train(model, loader, criterion, optimzer):
    model.train()
    train_loss, train_correct, train_total = 0.0, 0, 0
    for inputs ,labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimzer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimzer.step()
        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        train_correct += (preds == labels).sum().item()
        train_total +=labels.size(0)
    return train_loss / len(loader), train_correct / train_total

In [5]:
def test(model, loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return val_loss / len(loader), accuracy

In [49]:
num_epochs = 10
for epoch in range(num_epochs):
    train_loss, train_cc = train(model, train_loader, criterion, optimzer)
    test_loss, test_accuracy = test(model, train_loader, criterion)
    print(f'Epoch {epoch + 1} / {num_epochs}')
    print(f'Train Loss : {train_loss:.4f}, {train_cc:.4f}')
    print(f'Test Loss :{test_loss:.4f}, Train Accuracy:{test_accuracy:.4f}')

Epoch 1 / 10
Train Loss : 4.8677, 0.9395
Test Loss :4.2865, Train Accuracy:0.9469
Epoch 2 / 10
Train Loss : 4.9413, 0.9396
Test Loss :4.6733, Train Accuracy:0.9426
Epoch 3 / 10
Train Loss : 4.8728, 0.9387
Test Loss :4.2541, Train Accuracy:0.9446
Epoch 4 / 10
Train Loss : 4.9567, 0.9385
Test Loss :4.3502, Train Accuracy:0.9457
Epoch 5 / 10
Train Loss : 4.9790, 0.9382
Test Loss :4.1791, Train Accuracy:0.9474
Epoch 6 / 10
Train Loss : 4.8058, 0.9407
Test Loss :4.6471, Train Accuracy:0.9403
Epoch 7 / 10
Train Loss : 4.7596, 0.9393
Test Loss :4.5106, Train Accuracy:0.9436
Epoch 8 / 10
Train Loss : 4.8465, 0.9424
Test Loss :4.2318, Train Accuracy:0.9472
Epoch 9 / 10
Train Loss : 4.8457, 0.9409
Test Loss :4.2170, Train Accuracy:0.9488
Epoch 10 / 10
Train Loss : 4.8900, 0.9392
Test Loss :4.2275, Train Accuracy:0.9461


In [ ]:
def predict(model, image_path, transform):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0) 
    image =  image.to(device)
    with torch.no_grad():
        output = model(image)
        _, pred = torch.max(output, 1)
    return "dog" if pred.item() == 1 else "cat"

test_image = r"D:\AI\study1\data\test\test1\273.jpg"
print(f"Prediction: {predict(model, test_image, transform)}")